# Online Learning
We need to be able to continuously update our model and adapt to changes in the behavior of the attackers. To do so, we will be building an online learning model.

## Setup
Import the packages we need and read in the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3

with sqlite3.connect('logs/logs.db') as conn:
    logs_2018 = pd.read_sql(
        'SELECT * FROM logs WHERE datetime BETWEEN "2018-01-01" AND "2019-01-01";', 
        conn, parse_dates=['datetime'], index_col='datetime'
    )
    hackers_2018 = pd.read_sql(
        'SELECT * FROM attacks WHERE start BETWEEN "2018-01-01" AND "2019-01-01";', 
        conn, parse_dates=['start', 'end']
    ).assign(
        start_floor=lambda x: x.start.dt.floor('min'),
        end_ceil=lambda x: x.end.dt.ceil('min')
    )

Our functions for getting the X and y for our model:

In [ ]:
def get_X(log, day):
    """
    Get data we can use for the X
    
    Parameters:
        - log: The logs dataframe
        - day: A day or single value we can use as a datetime index slice
    
    Returns: 
        A `pandas.DataFrame` object
    """
    return pd.get_dummies(log.loc[day].assign(
        failures=lambda x: 1 - x.success
    ).query('failures > 0').resample('1min').agg(
        {'username': 'nunique', 'failures': 'sum'}
    ).dropna().rename(
        columns={'username': 'usernames_with_failures'}
    ).assign(
        day_of_week=lambda x: x.index.dayofweek, 
        hour=lambda x: x.index.hour
    ).drop(columns=['failures']), columns=['day_of_week', 'hour'])

def get_y(datetimes, hackers, resolution='1min'):
    """
    Get data we can use for the y (whether or not a hacker attempted a log in during that time).
    
    Parameters:
        - datetimes: The datetimes to check for hackers
        - hackers: The dataframe indicating when the attacks started and stopped
        - resolution: The granularity of the datetime. Default is 1 minute.
        
    Returns:
        `pandas.Series` of Booleans.
    """
    date_ranges = hackers.apply(
        lambda x: pd.date_range(x.start_floor, x.end_ceil, freq=resolution), 
        axis=1
    )
    dates = pd.Series(dtype='object')
    for date_range in date_ranges:
        dates = pd.concat([dates, date_range.to_series()])
    return datetimes.isin(dates)

def get_X_y(log, day, hackers):
    """
    Get the X, y data to build a model with.
    
    Parameters:
        - log: The logs dataframe
        - day: A day or single value we can use as a datetime index slice
        - hackers: The dataframe indicating when the attacks started and stopped
        
    Returns:
        X, y tuple where X is a `pandas.DataFrame` object
        and y is a `pandas.Series` object
    """
    X = get_X(log, day)
    y = get_y(X.reset_index().datetime, hackers)
    return X, y

We will work with the 2018 data to initially train our model, and then move to a monthly frequency to predict (and evaluate) each month and update the model afterwards (but before checking the next month).

In [ ]:
X_2018, y_2018 = get_X_y(logs_2018, '2018', hackers_2018)

## Stochastic Gradient Descent Classification
Using SGD, we will be able to build a logistic regression model that can be updated:

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from ml_utils.partial_fit_pipeline import PartialFitPipeline

model = PartialFitPipeline([
    ('scale', StandardScaler()), 
    ('sgd', SGDClassifier(
        random_state=10, max_iter=1000, tol=1e-3, loss='log', 
        average=1000, learning_rate='adaptive', eta0=0.01
    ))
]).fit(X_2018, y_2018)

As before, we can look at the coefficients for each of our features. The largest coefficient is `usernames_with_failures`:

In [ ]:
[(col, coef) for col, coef in zip(X_2018.columns, model.named_steps['sgd'].coef_[0])]

### Predict, Evaluate, and Update Model

#### Step 1: Gather test data
We will use January 2019:

In [ ]:
with sqlite3.connect('logs/logs.db') as conn:
    logs_2019 = pd.read_sql(
        """
        SELECT * 
        FROM logs 
        WHERE datetime BETWEEN "2019-01-01" AND "2020-01-01";
        """, conn, parse_dates=['datetime'], index_col='datetime'
    )
    hackers_2019 = pd.read_sql(
        """
        SELECT * 
        FROM attacks 
        WHERE start BETWEEN "2019-01-01" AND "2020-01-01";
        """, conn, parse_dates=['start', 'end']
    ).assign(
        start_floor=lambda x: x.start.dt.floor('min'),
        end_ceil=lambda x: x.end.dt.ceil('min')
    )

X_jan, y_jan = get_X_y(logs_2019, '2019-01', hackers_2019)

#### Step 2: Evaluate Performance
Recall is too low here:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_jan, model.predict(X_jan)))

We can visualize where we are on the ROC and precision-recall curves. Let's make a convenience function since we will be plotting these often:

In [ ]:
from ml_utils.classification import confusion_matrix_visual, plot_pr_curve, plot_roc

def plot_performance(model, X, y, threshold=None, title=None, show_target=True):
    """
    Plot the ROC, confusion matrix, and precision-recall curve side-by-side.
    
    Parameters:
        - model: The model object to use for prediction.
        - X: The features to pass in for prediction from the testing set.
        - y: The actuals to evaluate the prediction.
        - threshold: Value to use as threshold when predicting probabilities.
        - title: A title for the subplots.
        - show_target: Whether to show the target regions on the ROC/PR curve.
        
    Returns:
        Matplotlib `Axes` object.    
    """
    # make the subplots
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))

    # plot each visualization
    plot_roc(y, model.predict_proba(X)[:,1], ax=axes[0])
    confusion_matrix_visual(
        y, model.predict_proba(X)[:,1] >= (threshold or 0.5), 
        class_labels=[False, True], ax=axes[1]
    )
    plot_pr_curve(y, model.predict_proba(X)[:,1], ax=axes[2])

    # show the target regions if desired
    if show_target:
        axes[0].axvspan(0, 0.1, color='lightgreen', alpha=0.5)
        axes[0].axhspan(0.7, 1, color='lightgreen', alpha=0.5)
        axes[0].annotate(
            'region with acceptable\nFPR and TPR', 
            xy=(0.1, 0.7), xytext=(0.17, 0.65), 
            arrowprops=dict(arrowstyle='->')
        )

        axes[2].axvspan(0.7, 1, color='lightgreen', alpha=0.5)
        axes[2].axhspan(0.85, 1, color='lightgreen', alpha=0.5)
        axes[2].annotate(
            'region with acceptable\nprecision and recall', 
            xy=(0.7, 0.85), xytext=(0.3, 0.6), 
            arrowprops=dict(arrowstyle='->')
        )

        # mark the current performance
        tn, fn, fp, tp = [int(x.get_text()) for x in axes[1].texts]
        precision, recall, fpr = tp / (tp + fp), tp / (tp + fn), fp / (fp + tn)

        prefix = 'current performance' if not threshold else f'chosen threshold: {threshold:.2%}'

        axes[0].annotate(
            f'{prefix}\n- FPR={fpr:.2%}\n- TPR={recall:.2%}', 
            xy=(fpr, recall), xytext=(0.05, 0.45), 
            arrowprops=dict(arrowstyle='->')
        )
        axes[2].annotate(
            f'{prefix}\n- precision={precision:.2%}\n- recall={recall:.2%}', 
            xy=(recall, precision), xytext=(0.2, 0.85), 
            arrowprops=dict(arrowstyle='->')
        )

    # show the title if specified
    if title:
        plt.suptitle(title)

    return axes

Notice we are not quite where we need to be:

In [ ]:
axes = plot_performance(
    model, X_jan, y_jan, 
    title='Stochastic Gradient Descent Classifier '\
        '(Tested on January 2019 Data)'
)

##### Try picking a custom threshold based on acceptable TPR and FPR
We will be using the threshold with the precision-recall curve, but here is how to get it from the ROC curve:

In [ ]:
from ml_utils.classification import find_threshold_roc

threshold = find_threshold_roc(
    y_jan, model.predict_proba(X_jan)[:,1], 
    fpr_below=0.1, tpr_above=0.7
).max()
threshold

##### Try picking a custom threshold based on acceptable recall and precision
Note this is actually the same threshold:

In [ ]:
from ml_utils.classification import find_threshold_pr

threshold = find_threshold_pr(
    y_jan, model.predict_proba(X_jan)[:,1], 
    min_precision=0.85, min_recall=0.7
).max()
threshold

Using this threshold, we are able to move into the acceptable region:

In [ ]:
axes = plot_performance(
    model, X_jan, y_jan, threshold=threshold, 
    title='Stochastic Gradient Descent Classifier '\
        '(Tested on January 2019 Data)'
)

#### Step 4: Update the model to include true labels for previously predicted test data (Jan 2019)

In [ ]:
# update
model.partial_fit(X_jan, y_jan)

#### Step 5: Repeat process (this time we use Feb 2019)

In [ ]:
X_feb, y_feb = get_X_y(logs_2019, '2019-02', hackers_2019)

print(classification_report(y_feb, model.predict_proba(X_feb)[:,1] >= threshold))

Notice the model is better and now a larger section of both the ROC curve and the precision-recall curve fall in the acceptable performance region:

In [ ]:
axes = plot_performance(
    model, X_feb, y_feb, threshold=threshold,
    title='Stochastic Gradient Descent Classifier '\
        '(Tested on February 2019 Data)'
)

## Putting our model to the test
In the time that we've been building all these models, March 2019 has come and gone. Our stakeholders are done waiting around for us and want results. Let's show our performance.

### Step 1: Update with the February 2019 data first

In [ ]:
model.partial_fit(X_feb, y_feb)

### Step 2: Predict the March data for the stakeholders and send it off

In [ ]:
X_march, y_march = get_X_y(logs_2019, '2019-03', hackers_2019)
march_2019_preds = model.predict_proba(X_march)[:,1] >= threshold

### Step 3: Evaluation
The subject area experts examined our predictions and returned the following after checking our predictions. We are meeting our metrics:

In [ ]:
# classification report
from sklearn.metrics import classification_report
print(classification_report(y_march, march_2019_preds))

We were able to adapt to new data and still meet our performance requirements:

In [ ]:
axes = plot_performance(
    model, X_march, y_march, threshold=threshold,
    title='Stochastic Gradient Descent Classifier '\
        '(Tested on March 2019 Data)'
)

<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./4-supervised_anomaly_detection.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_11/exercise_1.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../../ch_12/README.md">
            <button>Chapter 12 &#8594;</button>
        </a>
    </div>
</div>
<hr>